# Objetivo

Conhecer as variáveis iniciais através de uma exploratória básica para compreender a proposta introdutória do que será desenvolvido durante o projeto. 

O projeto será baseado no meu TCC, portanto, ainda existem alterações sobre a database que será utilizada. Caso ocorram alterações, irei indicar o momento e construir novamente as análises iniciais.

O domínio dos dados é relacionado a: Preço de energia elétrica

# To Do
- Verificar características da base [Volume, variáveis]
- Verificar as variáveis [Tipos e valores]
- - Medidas de posição, dispersão e forma



In [0]:
import sys

import os

In [0]:
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [0]:
from pyspark.sql import functions as F

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# Importação das Bibliotecas e Parâmetros de Otimização

# %matplotlib inline
# sns.set(style="whitegrid", font_scale=1.2)
# plt.rcParams['font.family'] = 'Arial'
# plt.rcParams['font.size'] = '14'
# plt.rcParams['figure.figsize'] = [10, 5]
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_columns', 100)
# #pd.set_option('display.max_colwidth', None)
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
# np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
# warnings.filterwarnings('ignore')
# warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Leitura das bases

- Com o uso da ferramenta na versão limitada, foi necessário construir uma tabela dentro do catálogo para realizar a leitura dela. 
- O dataset também está disponível no diretório 'data' do projeto.
- A base está resumida e é uma versão prévia com poucas variáveis apenas para estabelecer a proposta do projeto que será desenvolvido. 

In [0]:
df = spark.table('workspace.default.electricity_cost_dataset')

In [0]:
df.display()


# Conhecendo as variáveis
Ao todo temos 10.000 linhas, 0 linhas duplicadas e disponíveis em 9 variáveis.

- As nomenclaturas de variáveis não está padronizado. Não há uma taxonomia definida. 
- As nomenclaturas indicam diretamente os valores que 'caracterizam'. Não há um dicionário.
- Há uma atenção para as variáveis 'structure type' e 'resident count', qualitativa nominal e quantitativa discreta. O restante é contínua.
- As variáveis podem ter pontos de atenção devido ao comportamento diferente dos tipos de 'structure type'. Talvez, ao filtrar especificamente cada uma das categorias, os dados se tornem mais 'normais'.

In [0]:
print(f'Há um total de {df.count()} registros')

In [0]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(f"Há um total de {duplicate_rows} registros duplicados")

In [0]:
df.dtypes

In [0]:
# Selecionando somente as colunas numéricas
numerical_cols = [col_name for col_name, dtype in df.dtypes if dtype in ['bigint', 'big' 'double']]

In [0]:
# Selecionando somente as colunas categóricas
categorical_cols = [col_name for col_name, dtype in df.dtypes if dtype == 'string']

## Avaliação de medidas

- Devido as características das variáveis contínuas é notável que há uma dispersão maior entre elas. Não há nenhuma extrapolação visível inicialmente
- Há muitos outliers na 'resident count'.  


In [0]:
# Verificando as estatísticas das colunas 
for var in numerical_cols:
    df.select(var).summary().display()

In [0]:
missing_values = df.select([F.when(F.col(c).isNull(), 1).otherwise(0).alias(c) for c in df.columns])\
                  .agg(*[F.sum(F.col(c)).alias(c) for c in df.columns])

missing_values.display()

In [0]:
df_pandas = df.toPandas()

# Ajuste no tamanho da figura com base no número de colunas numéricas
plt.figure(figsize=(10, 5 * len(numerical_cols)))

# Boxplot, Violin Plot e Histograma para cada variável numérica (visualização vertical)
for i, col in enumerate(numerical_cols):
    # Boxplot
    plt.subplot(len(numerical_cols), 3, 3*i + 1)
    sns.boxplot(y=df_pandas[col])
    plt.title(f'Boxplot - {col}')
    
    # Violin Plot
    plt.subplot(len(numerical_cols), 3, 3*i + 2)
    sns.violinplot(y=df_pandas[col])
    plt.title(f'Violin Plot - {col}')
    
    # Distribuição com Histograma
    plt.subplot(len(numerical_cols), 3, 3*i + 3)
    sns.histplot(df_pandas[col], kde=True)
    plt.title(f'Distribuição de {col}')

# Ajuste da disposição dos subgráficos para evitar sobreposição
plt.tight_layout()
plt.show()

In [0]:
structures_list = df.select('structure type').distinct().collect()

structures_list = [row['structure type'] for row in df.select('structure type').distinct().collect()]

structures_list


In [0]:
for structure in structures_list:
    df_filtered = df.filter(F.col('structure type') == structure)
    df_pandas = df_filtered.toPandas()

    # Ajuste no tamanho da figura com base no número de colunas numéricas
    plt.figure(figsize=(10, 5 * len(numerical_cols)))
    plt.suptitle(f'Structure Type: {structure}', fontsize=16, y=1.02)

    # Boxplot, Violin Plot e Histograma para cada variável numérica (visualização vertical)
    for i, col in enumerate(numerical_cols):
        # Boxplot
        plt.subplot(len(numerical_cols), 3, 3*i + 1)
        sns.boxplot(y=df_pandas[col])
        plt.title(f'Boxplot - {col}')

        # Violin Plot
        plt.subplot(len(numerical_cols), 3, 3*i + 2)
        sns.violinplot(y=df_pandas[col])
        plt.title(f'Violin Plot - {col}')

        # Distribuição com Histograma
        plt.subplot(len(numerical_cols), 3, 3*i + 3)
        sns.histplot(df_pandas[col], kde=True)
        plt.title(f'Distribuição de {col}')

    # Ajuste da disposição dos subgráficos para evitar sobreposição
    plt.tight_layout()
    plt.show()

In [0]:
for col_name in categorical_cols:
    df.groupBy(col_name).count().display()

In [0]:
df_pandas = df.toPandas()

plt.figure(figsize=(10, 5 * len(categorical_cols)))

# Gráfico de barras para variáveis categóricas com % centralizada dentro da barra
for i, col in enumerate(categorical_cols):
    plt.subplot(len(categorical_cols), 1, i+1)
    ax = sns.countplot(x=col, data=df_pandas)
    plt.title(f'Distribuição das Categorias - {col}')
    plt.xlabel('Categorias')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    
    total = len(df_pandas)
    for p in ax.patches:
        count = int(p.get_height())
        percent = 100 * count / total
        ax.annotate(f'{percent:.1f}%', 
                    (p.get_x() + p.get_width() / 2, p.get_height() / 2), 
                    ha='center', va='center', fontsize=11, color='white', fontweight='bold')

plt.tight_layout()
plt.show()

In [0]:
# Cálculo da matriz de correlação para todas as combinações de colunas numéricas
df_pandas = df.select(numerical_cols).toPandas()
correlation_matrix = df_pandas.corr()

# Visualização da correlação usando Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()